In [1]:
import os
import json
from model import modelRecruitment

with open('../config.json','rb') as file:
    config = json.load(file)

path_to_model = os.getcwd() + "/model"
path_to_model

'/Users/yeremi-intiva/Documents/Local/de/py-pg/data-stream-processing/consumer/model'

In [2]:
from pymongo import MongoClient

host = config['mongodb']['host']
port = config['mongodb']['port']
user = config['mongodb']['user']
password = config['mongodb']['password']

try:
    server = MongoClient(f'mongodb://{user}:{password}@{host}:{port}/')
    db = server.admin
    server_status = db.command("ping")
    print("MongoDB connection successful:", server_status)

    databases = server.list_database_names()
    print("Databases:", databases)

except Exception as e:
    print("An error occurred:", e)

MongoDB connection successful: {'ok': 1.0}
Databases: ['admin', 'config', 'ftde02', 'ftde03', 'ftde3', 'local']


In [3]:
import pandas as pd

def predict_row(row):
    data_predict = row.to_dict()
    return modelRecruitment.runModel(data_predict, path_to_model)

db = server["ftde3"]
collection = db["kelompok2_data_recruitment_selection_raw"]

data = pd.DataFrame(list(collection.find()))

columns_to_drop = ['_id', 'CandidateID', 'Name', 'ApplicationDate', 'InterviewDate']
data_predict = data.drop(columns=columns_to_drop, axis=1)

data['Predict'] = data_predict.apply(predict_row, axis=1)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.3.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.3.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.3.0 when using version 1.5.1. This might lead to breaking

In [4]:
new_data = data.drop(columns=['_id']).to_dict(orient='records')

db = server["ftde3"]
collection = db["kelompok2_data_recruitment_selection"]

collection.insert_many(new_data)

InsertManyResult([ObjectId('676d15360f0dfd89af0ae180'), ObjectId('676d15360f0dfd89af0ae181'), ObjectId('676d15360f0dfd89af0ae182'), ObjectId('676d15360f0dfd89af0ae183'), ObjectId('676d15360f0dfd89af0ae184'), ObjectId('676d15360f0dfd89af0ae185'), ObjectId('676d15360f0dfd89af0ae186'), ObjectId('676d15360f0dfd89af0ae187'), ObjectId('676d15360f0dfd89af0ae188'), ObjectId('676d15360f0dfd89af0ae189'), ObjectId('676d15360f0dfd89af0ae18a'), ObjectId('676d15360f0dfd89af0ae18b'), ObjectId('676d15360f0dfd89af0ae18c'), ObjectId('676d15360f0dfd89af0ae18d'), ObjectId('676d15360f0dfd89af0ae18e'), ObjectId('676d15360f0dfd89af0ae18f'), ObjectId('676d15360f0dfd89af0ae190'), ObjectId('676d15360f0dfd89af0ae191'), ObjectId('676d15360f0dfd89af0ae192'), ObjectId('676d15360f0dfd89af0ae193'), ObjectId('676d15360f0dfd89af0ae194'), ObjectId('676d15360f0dfd89af0ae195'), ObjectId('676d15360f0dfd89af0ae196'), ObjectId('676d15360f0dfd89af0ae197'), ObjectId('676d15360f0dfd89af0ae198'), ObjectId('676d15360f0dfd89af0ae1

In [11]:
collection = db["default"]

data = pd.DataFrame(list(collection.find()))

print(not data.empty)

False
